In [ ]:
from datetime import datetime
import pandas as pd
from sqlalchemy import create_engine

# Database connection setup
db_config = {
    'host': 'localhost',
    'database': 'postgres',
    'user': 'postgres',
    'password': 'kaviyam123',
    'port': '5432'
}
connection_string = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
engine = create_engine(connection_string)

# Step 1: Load data
query = "SELECT * FROM overall_merged_data_31_12;"
df = pd.read_sql(query, con=engine)

# Convert dates to datetime
df['Policy Start Date'] = pd.to_datetime(df['Policy Start Date'], errors='coerce')
df['Policy End Date'] = pd.to_datetime(df['Policy End Date'], errors='coerce')

# Step 2: Remove invalid rows in 'Total Premium Payable'
df = df[~df['Total Premium Payable '].isnull()]  # Remove null values
df = df[~df['Total Premium Payable '].astype(str).str.isalpha()]  # Remove categorical values
df = df[df['Total Premium Payable '] != 0]  # Remove rows with 0

# Step 3: Handle NULL values in BOOKED column
today = pd.Timestamp(datetime.now().date())  # Current date

def update_booked(row):
    if pd.isnull(row['booked']):
        if row['Type'] == 'C':
            if pd.notnull(row['Policy End Date']):
                return 1 if row['Policy End Date'] <= today else '-'
        elif row['Type'] in ['A', 'B']:
            if pd.notnull(row['Policy End Date']):
                return 0 if row['Policy End Date'] <= today else '-'
    return row['booked']

df['booked'] = df.apply(update_booked, axis=1)

# Step 4: Deduplication and prioritization
duplicates = df[df.duplicated(subset=['Policy No', 'Policy Start Date', 'Policy End Date'], keep=False)]

def prioritize_rows(group):
    group = group.assign(null_count=group.isnull().sum(axis=1))
    group = group.sort_values(by=['null_count', 'booked'], ascending=[True, False])
    return group.iloc[0]

cleaned_duplicates = (
    duplicates.groupby(['Policy No', 'Policy Start Date', 'Policy End Date'])
    .apply(prioritize_rows)
    .reset_index(drop=True)
)

df_cleaned = pd.concat([df, cleaned_duplicates]).drop_duplicates(subset=['Policy No', 'Policy Start Date', 'Policy End Date'], keep='last')

# Step 5: Correct BOOKED values based on Type
correction_count = 0

def correct_booked(group):
    global correction_count
    type_a = group[group['Type'] == 'A']
    type_b = group[group['Type'] == 'B']
    
    # Check if both Type A and Type B exist for the same Policy No
    if not type_a.empty and not type_b.empty:
        # Check if BOOKED is 0 for Type A
        if type_a.iloc[0]['booked'] == 0:
            correction_count += 1
            # Update BOOKED to 1 for Type A
            group.loc[group['Type'] == 'A', 'booked'] = 1
    return group

type_a_b = df_cleaned[df_cleaned['Type'].isin(['A', 'B'])]
type_a_b_grouped = type_a_b.groupby('Policy No')

df_cleaned = type_a_b_grouped.apply(correct_booked).reset_index(drop=True)

# Output the number of corrections made
print(f"Number of corrections made: {correction_count}")

# Save the cleaned dataset back to the database
df_cleaned.to_sql('cleaned_overall_merged_data_31_12', con=engine, if_exists='replace', index=False)

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
from sqlalchemy import create_engine

# Database connection setup
db_config = {
    'host': 'localhost',
    'database': 'postgres',
    'user': 'postgres',
    'password': 'kaviyam123',
    'port': '5432'
}
connection_string = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
engine = create_engine(connection_string)

# Step 1: Load data
query = "SELECT * FROM overall_merged_data_31_12;"
df = pd.read_sql(query, con=engine)

# Convert dates to datetime
df['Policy Start Date'] = pd.to_datetime(df['Policy Start Date'], errors='coerce')
df['Policy End Date'] = pd.to_datetime(df['Policy End Date'], errors='coerce')

# Step 2: Remove invalid rows in 'Total Premium Payable'
df = df[~df['Total Premium Payable '].isnull()]  # Remove null values
df = df[~df['Total Premium Payable '].astype(str).str.isalpha()]  # Remove categorical values
df = df[df['Total Premium Payable '] != 0]  # Remove rows with 0

# Ensure the 'booked' column is explicitly cast to a string type
if 'booked' not in df.columns or df['booked'].isnull().all():
    df['booked'] = None  # If the column doesn't exist or is completely null, initialize it
df['booked'] = df['booked'].astype('str')  # Convert to string to handle assignments of '0', '1', '-'

today = pd.Timestamp.now().normalize()

def update_booked(group):
    # Sort policies by start date within the group
    group = group.sort_values(by='Policy Start Date')
    
    for i in range(len(group) - 1):
        current_policy = group.iloc[i]
        next_policy = group.iloc[i + 1]
        
        # Only process rows where booked is NULL or 'None'
        if pd.isnull(current_policy['booked']) or current_policy['booked'] == 'None':
            # Check if the next policy starts the day after the current one ends
            if next_policy['Policy Start Date'] == current_policy['Policy End Date'] + timedelta(days=1):
                if next_policy['Policy Start Date'] > today:
                    group.loc[current_policy.name, 'booked'] = '1'
                else:
                    group.loc[current_policy.name, 'booked'] = '1'
            elif next_policy['Policy Start Date'] > current_policy['Policy End Date'] + timedelta(days=1):
                group.loc[current_policy.name, 'booked'] = '1'
    
    # Handle the last policy for the customer
    last_policy = group.iloc[-1]
    if pd.isnull(last_policy['booked']) or last_policy['booked'] == 'None':
        if last_policy['Policy End Date'] >= today:
            group.loc[last_policy.name, 'booked'] = '-'
        else:
            group.loc[last_policy.name, 'booked'] = '0'
    
    return group

# Apply the function to each group of policies
df = df.groupby('Policy No').apply(update_booked).reset_index(drop=True)

# Step 4: Handle duplicates and prioritize
duplicates = df[df.duplicated(subset=['Policy No', 'Policy Start Date', 'Policy End Date'], keep=False)]

def prioritize_rows(group):
    group = group.assign(null_count=group.isnull().sum(axis=1))
    group = group.sort_values(by=['null_count', 'booked'], ascending=[True, False])
    return group.iloc[0]

cleaned_duplicates = (
    duplicates.groupby(['Policy No', 'Policy Start Date', 'Policy End Date'])
    .apply(prioritize_rows)
    .reset_index(drop=True)
)

df_cleaned = pd.concat([df, cleaned_duplicates]).drop_duplicates(subset=['Policy No', 'Policy Start Date', 'Policy End Date'], keep='last')

def correct_booked(group):
    group = group.sort_values(by='Policy Start Date')
    for i in range(len(group) - 1):
        current_policy = group.iloc[i]
        next_policy = group.iloc[i + 1]
        
        # Check if the current policy is marked as '0' but satisfies the condition
        if current_policy['booked'] == '0':
            # Check if the next policy starts the day after the current one ends
            if next_policy['Policy Start Date'] == current_policy['Policy End Date'] + timedelta(days=1):
                if next_policy['Policy Start Date'] > today:
                    group.loc[current_policy.name, 'booked'] = '1'
                else:
                    group.loc[current_policy.name, 'booked'] = '1'
            # Check if the next policy starts after a gap
            elif next_policy['Policy Start Date'] > current_policy['Policy End Date'] + timedelta(days=1):
                if next_policy['Policy Start Date'] > today:
                    group.loc[current_policy.name, 'booked'] = '1'
    
    return group

# Apply the correction to each group of policies
df_cleaned = df_cleaned.groupby('Policy No').apply(correct_booked).reset_index(drop=True)

# Save the cleaned dataset back to the database
df_cleaned.to_sql('cleaned_overall_merged_base_pr_data', con=engine, if_exists='replace', index=False)

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
from sqlalchemy import create_engine

# Database connection setup
db_config = {
    'host': 'localhost',
    'database': 'postgres',
    'user': 'postgres',
    'password': 'kaviyam123',
    'port': '5432'
}
connection_string = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
engine = create_engine(connection_string)

# Step 1: Load data
query = "SELECT * FROM overall_merged_data_31_12;"
df = pd.read_sql(query, con=engine)

# Convert dates to datetime
df['Policy Start Date'] = pd.to_datetime(df['Policy Start Date'], errors='coerce')
df['Policy End Date'] = pd.to_datetime(df['Policy End Date'], errors='coerce')

# Step 2: Remove invalid rows in 'Total Premium Payable'
df = df[~df['Total Premium Payable '].isnull()]  # Remove null values
df = df[~df['Total Premium Payable '].astype(str).str.isalpha()]  # Remove categorical values
df = df[df['Total Premium Payable '] != 0]  # Remove rows with 0

# Ensure the 'booked' column is explicitly cast to a string type
if 'booked' not in df.columns or df['booked'].isnull().all():
    df['booked'] = None  # If the column doesn't exist or is completely null, initialize it
df['booked'] = df['booked'].astype('str')  # Convert to string to handle assignments of '0', '1', '-'

today = pd.Timestamp.now().normalize()

def update_booked(group):
    # Sort policies by start date within the group
    group = group.sort_values(by='Policy Start Date')
    
    for i in range(len(group) - 1):
        current_policy = group.iloc[i]
        next_policy = group.iloc[i + 1]
        
        # Only process rows where booked is NULL or 'None'
        if pd.isnull(current_policy['booked']) or current_policy['booked'] == 'None':
            # Check if the next policy starts the day after the current one ends
            if next_policy['Policy Start Date'] == current_policy['Policy End Date'] + timedelta(days=1):
                if next_policy['Policy Start Date'] > today:
                    group.loc[current_policy.name, 'booked'] = '1'
                else:
                    group.loc[current_policy.name, 'booked'] = '1'
            elif next_policy['Policy Start Date'] > current_policy['Policy End Date'] + timedelta(days=1):
                group.loc[current_policy.name, 'booked'] = '1'
    
    # Handle the last policy for the customer
    last_policy = group.iloc[-1]
    if pd.isnull(last_policy['booked']) or last_policy['booked'] == 'None':
        if last_policy['Policy End Date'] >= today:
            group.loc[last_policy.name, 'booked'] = '-'
        else:
            group.loc[last_policy.name, 'booked'] = '0'
    
    return group

# Initialize an empty list to store processed groups
processed_groups = []

# Group by 'Policy No' and process each group manually
for policy_no, group in df.groupby('Policy No'):
    processed_group = update_booked(group)
    processed_groups.append(processed_group)

# Combine all processed groups back into a single DataFrame
df = pd.concat(processed_groups).reset_index(drop=True)

# Step 4: Handle duplicates and prioritize
duplicates = df[df.duplicated(subset=['Policy No', 'Policy Start Date', 'Policy End Date'], keep=False)]

def prioritize_rows(group):
    group = group.assign(null_count=group.isnull().sum(axis=1))
    group = group.sort_values(by=['null_count', 'booked'], ascending=[True, False])
    return group.iloc[0]

cleaned_duplicates = (
    duplicates.groupby(['Policy No', 'Policy Start Date', 'Policy End Date'])
    .apply(prioritize_rows)
    .reset_index(drop=True)
)

df_cleaned = pd.concat([df, cleaned_duplicates]).drop_duplicates(subset=['Policy No', 'Policy Start Date', 'Policy End Date'], keep='last')

def correct_booked(group):
    group = group.sort_values(by='Policy Start Date')
    for i in range(len(group) - 1):
        current_policy = group.iloc[i]
        next_policy = group.iloc[i + 1]
        
        # Check if the current policy is marked as '0' but satisfies the condition
        if current_policy['booked'] == '0':
            # Check if the next policy starts the day after the current one ends
            if next_policy['Policy Start Date'] == current_policy['Policy End Date'] + timedelta(days=1):
                if next_policy['Policy Start Date'] > today:
                    group.loc[current_policy.name, 'booked'] = '1'
                else:
                    group.loc[current_policy.name, 'booked'] = '1'
            # Check if the next policy starts after a gap
            elif next_policy['Policy Start Date'] > current_policy['Policy End Date'] + timedelta(days=1):
                if next_policy['Policy Start Date'] > today:
                    group.loc[current_policy.name, 'booked'] = '1'
    
    return group

# Apply the correction to each group of policies
df_cleaned = df_cleaned.groupby('Policy No').apply(correct_booked).reset_index(drop=True)

# Save the cleaned dataset back to the database
df_cleaned.to_sql('cleaned_overall_merged_base_pr_data', con=engine, if_exists='replace', index=False)